In [2]:
import pandas as pd

In [3]:
ranking_file =  "metadata\CSI_imdb_ranking_synopsis.csv"

rankings = pd.read_csv(ranking_file, header=None)

rankings.head()

,0,1
0,For Warrick,1. CSI: Crime Scene Investigation (2000–2015) ...
1,One to Go,2. CSI: Crime Scene Investigation (2000–2015) ...
2,Living Doll,3. CSI: Crime Scene Investigation (2000–2015) ...
3,Girls Gone Gadgets,4. CSI: Crime Scene Investigation (2000–2015) ...
4,Gum Drops,5. CSI: Crime Scene Investigation (2000–2015) ...


In [4]:
test_cell = rankings[1][0]

In [5]:
test_cell

"1. CSI: Crime Scene Investigation (2000–2015) Episode: For Warrick (2008) TV-14 | 45 min | Crime, Drama, Mystery 9.0 Rate this CSIs past and present come together at the news of Warrick's death. But will Warrick's real killer successfully frame another police officer for his murder? Director: Richard J. Lewis | Stars: William Petersen , Marg Helgenberger , Gary Dourdan , George Eads Votes: 1,083"

In [101]:
def cell_splitter(test_cell):
    test_cell = test_cell.replace('PG','14')
    test_cell = test_cell.replace('MA','14')
    rank, *r = test_cell.split('. CSI: Crime Scene Investigation (2000–2015) ')
    season, *r = r[0].split('Episode: ')
    name, *r = r[0].split(' TV-14 | ')
    r = r[0].split('Crime, Drama, Mystery ')[1:]
    rating, *r = r[0].split(' Rate this ')
    # synopsis, *r = r[0].split(' Director: ')
    # director, *r = r[0].split(' | Stars: ')
    # stars, *r = r[0].split(' Votes: ')
    # votes = r[0]
    # name =  test_cell.split()
    # print(rank, name, rating, sep='\n')
    return [rank, name, rating]



In [102]:
cell_splitter(test_cell)

['1', 'For Warrick (2008)', '9.0']

In [118]:
new_rankings = pd.DataFrame(rankings[1].apply(cell_splitter).to_list(), columns=['rank_order', 'epsisode', 'rank'])

In [119]:
new_rankings.to_csv("metadata\CSI_imdb_ranking.csv")